# Генетический алгоритм на примере задачи разбиения

## Постановка задачи

Задача: разбить исходный массив на два массива таким образом, чтобы суммы элементов полученных массивов были равны.

Формальное описание задачи: дан исходный массив $W = [w_1, w_2, w_3, ..., w_n]$, найти такие массивы $A\subset W, B = W \backslash A$, что $\sum_{a \in A} a = \sum_{b \in B} b$.

## Описание решения

Схема генетического алгоритма в общем виде выглядит следующим образом:
- Кодирование
- Генерация
- Выбор мутирующих особей
- Мутация
- Выбор скрещивающихся особей
- Скрещивание
- Отбор

Опишем каждый шаг данной схемы применимо к нашей задаче. На этапе **кодирования** мы определяем то, как будет выглядеть структура наших «особей». В данном случае она будут являться булевым вектором длины вектора $W$, где каждый элемент будет обозначать принадлежность к одному из искомых массивов (True - массив A, False - массив B).

На этапе **генерации** мы породим набор из 100 случайно сгенерированных особей. Они будут являться первым поколением особей нашего алгоритма. Все следующие операции будут выполняться в цикле, количество итераций в котором будет равняться необходимому нам количеству поколений.

На этапе **выбора мутирующих особей** мы выберем некое случайное количество мутирующих особей в текущем поколении, а на этапе **мутации** изменим у каждой из них случаный элемент массива на противоположный.

Дальше необходимо выполнить **выбор скрещивающихся особей**. Применим функцию приспособленности (которая оценивает насколько хорошо особь «решает» поставленную задачу) к каждой особи и отсортируем их в порядке уменьшения приспособленности. Функция приспособленности устроена таким образом, что возвращает число от 0 (полная неприспособленность) до 1 (максимальная приспособленность - особь представляет собой правильное решение задачи). Пары составляем из соседних особей в отсортированном ряду без наложения (т.е. парами будут особи с порядковыми номерами 1 и 2, 3 и 4 и т.д.).

Следующим этапом производим **скрещивание** пар особей. В результате скрещивания двух особей будет получаться одна новая особь, элементы структуры которой будут случайным образом заимствованы от каждого из родителей. Более формально это можно описать следующим образом: элемент структуры с индексом $i$ дочерней особи будет случайно выбран из двух родительских элементов с индексом $i$.

На этапе **отбора** мы снова отсортируем всех особей в порядке уменьшения их приспособленности и удалим 50 наименее приспособленных особей.

## Реализация алгоритма

In [1]:
import random

In [2]:
def mutation(individ):
    '''
    Процедура мутации - выбирает случаный элемент особи и
    меняет его значение на противоположное
    '''
    # индекс мутирующего элемента
    i = random.randint(0, len(W) - 1)
    individ[i] = not individ[i]

In [3]:
def fitness(individ):
    '''
    Функция приспособленности. Принимает на вход особь и 
    возращает её коэффициент приспособленности - число от 0 
    (полная неприспособленность) до 1 (максимальная приспособленность)
    '''
    A_sum = 0
    B_sum = 0
    
    for i in range(len(individ)):
        if individ[i]:
            A_sum += W[i]
        else:
            B_sum += W[i]
            
    return (1 + abs(A_sum - B_sum)) ** (-1)

In [4]:
def crossover(father, mother):
    '''
    Функция скрещивания особей. Принимает на вход две родительские
    особи и возращает дочернюю особь, элементы структуры которой 
    будут случайным образом заимствованы от каждого из родителей
    '''
    child = []
    for i in range(len(father)):
        flag = bool(random.getrandbits(1))
        if flag:
            child.append(father[i])
        else:
            child.append(mother[i])
    return child

In [5]:
def evaluate_result(individ):
    '''
    Функция проверки результата работы алгоритма. Разбивает исходный
    массив на два массива по правилу, которое содержит переданная в
    функцию особь, и возвращает суммы элементов этих массивов. Если
    суммы равны, то возращает полученные массивы, иначе возвращает 0
    '''
    A = []
    B = []
    
    A_sum = 0
    B_sum = 0
    
    for i in range(len(individ)):
        if individ[i]:
            A.append(W[i])
            A_sum += W[i]
        else:
            B.append(W[i])
            B_sum += W[i]
            
    if (A_sum == B_sum):
        return (A, B)
    else:
        return 0

In [6]:
def make_evolution(W, count_of_generations):
    '''
    Функция генетического алгоритма. Принимает на вход массив, который 
    необходимо разбить и количество поколений (циклов генетического
    алгоритма). 
    '''
    # массив с особями
    population = []
    
    # первичное заполнение
    for i in range(100):
        population.append([bool(random.getrandbits(1)) for i in range(len(W))])
        
    for i in range(count_of_generations):
        # число мутирующих особей
        mutation_number = random.randint(0, len(population) - 1)
        
        # индексы мутирующих особей
        mutation_indices = random.sample(range(len(population)), mutation_number)

        # процедура мутации
        for index in mutation_indices:
            mutation(population[index])

        # сортировка популяции
        population = sorted(population, key=fitness, reverse=True)

        # скрещивание
        children = []
        for i in range(0, len(population), 2):
            if (i + 1 <= len(population)):
                children.append(crossover(population[i], population[i + 1]))
        population += children

        # сортировка и отбор
        population = sorted(population, key=fitness, reverse=True)
        population = population[:-50]
        
    # анализ лучшей особи и вывод результата
    best_result = evaluate_result(population[0])
    if best_result == 0:
        print('Алгоритм не нашёл решения задачи')
    else:
        print('A:', best_result[0])
        print('B:', best_result[1])

## Тестирование

In [7]:
W = [12, 3, 2, 5, 10, 12, 1, 10, 10, 8, 6, 6, 20, 5, 10, 6, 3, 3]

In [8]:
count_of_generations = 1000

In [9]:
make_evolution(W, count_of_generations)

A: [5, 12, 10, 6, 6, 5, 10, 6, 3, 3]
B: [12, 3, 2, 10, 1, 10, 8, 20]
